# Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

from pandas import ExcelWriter
import xlsxwriter

In [2]:
data=pd.read_excel(r'C:\Users\Umar Muhammad\Dropbox\TO3 South\DHIS2\Gap Analysis\2021\Q4\October 2021\gaps_july-sept\melt\clean_data_south.xlsx')

In [3]:
imm=data[['Facility Name', 'State', 'LGA', 'Ward','Month_Year','Penta. 1 given', 'Penta 3 given']]
imm=imm.fillna(0)

In [4]:
penta_dropout = (imm["Penta. 1 given"] - imm["Penta 3 given"]).div(imm['Penta. 1 given'])

imm['penta_dropout']=penta_dropout

In [5]:
imm['dropout_rate']=pd.Series(["{0:.2f}%".format(val * 100) for val in imm['penta_dropout']], index = imm.index)

In [6]:
imm1=imm

## Validations

In [7]:
conditions = [(imm['penta_dropout'] < 0)]
choices = ['negative dropout']
imm['Immunization_Remarks-1'] = np.select(conditions,choices, default='')

In [8]:
conditions = [(imm['Penta. 1 given'] == 0) & (imm['Penta 3 given'] == 0)]
choices = ['no reports']
imm['Immunization_Remarks-2'] = np.select(conditions,choices, default='')

In [11]:
conditions = [(imm['penta_dropout'] >= 0.101 )]
choices = ['high dropout rate']
imm['Immunization_Remarks-3'] = np.select(conditions,choices, default='')

In [12]:
Remarks= imm[['Facility Name', 'State', 'LGA', 'Ward', 'Month_Year','Immunization_Remarks-1','Immunization_Remarks-2','Immunization_Remarks-3']]

In [13]:
melt = Remarks.melt(id_vars=['Facility Name','LGA', 'Ward', 'Month_Year'])

In [14]:
melt = melt.drop(index=melt[melt['value']==''].index)
melt = melt.drop(index=melt[melt['value']=='  State'].index)

In [15]:
melt.to_excel('output/Immunization_remarks_all.xls', index = False)